In [ ]:
from MDSetup import MDSetup

# MDSetup

This notebook enables the setup and analysis of MD simulations using GROMACS in an automated fashion. The idea is to introduce the **FAIR** principles:

1. **F**indable ( metadata and data should be easy to find for humans and computers )
2. **A**ccessible ( (meta)data should accessable by standardised communications protocol, e.g.: file formats )
3. **I**nteroperable ( data need to interoperate with applications or workflows for analysis, storage, and processing )
4. **R**eusable ( (meta)data should be well-described to replicate it in different settings )

There is the possiblity to provide data and GROMACS compatible parameter files, or to build a system and write all necessary input using GROMACS and moleculegraph.

1) Read in the YAML files to define the system and simulation/sampling settings.

In [ ]:
gromacs_setup = MDSetup( system_setup = "input/gromacs/setup.yaml", 
                        simulation_default = "input/gromacs/defaults.yaml",
                        simulation_ensemble = "input/gromacs/ensemble.yaml",
                        submission_command = "qsub"
                     )

## Creating force field and molecule files

In this section, MDSetup and moleculegraph (https://github.com/maxfleck/moleculegraph) is used to setup a force field and molecule files, which are needed to build and simulate the system

In [ ]:
gromacs_setup.write_topology( verbose = True )

## Setting up a simulation pipeline

In this section the possibility to setup a simulation folder, along with a simulation pipeline using several ensembles, is provided.

1) Setup simulation and build initial system (if not provided)


In [ ]:
# Define the simulation folder
simulation_folder = "md_thermo"

# Define the ensembles that should be simulated (definition what each ensemble means is provided in yaml file)
ensembles = [ "em", "npt" ] 

# Define the simulation time per ensemble in nano seconds (for em the number of iterations is provided in the ensemble yaml)
simulation_times = [ 0, 0.1 ]

# Define initial systems, in case the simulation should be continued from a prior simulation.
# In that case, provide one initial structure for each temperature & pressure state.
# If the simulation should start from an initial configuration, provide an empty list.
initial_systems = [ ]

# Provide kwargs that should be passed into the input template directly
input_kwargs = {  }

# Define number of copies
copies = 1

# Define if the inital system should build locally or with the cluster
on_cluster = False

# Define the starting number for the first ensemble ( 0{off_set}_ensemble )
off_set = 0

# In case templates for job submission or any other templates are edited
# Any keyword argument can be passed to this function call
gromacs_setup.prepare_simulation( 
    folder_name = simulation_folder,
    ensembles = ensembles,
    simulation_times = simulation_times,
    initial_systems = initial_systems,
    input_kwargs = input_kwargs,
    copies = copies,
    on_cluster = on_cluster,
    off_set = off_set 
)

2) Submit jobs to cluster

In [ ]:
# Submit the simulations
gromacs_setup.submit_simulation()

## Extract sampled properties

In [ ]:
# Extract properties from LAMMPS and analyse them

# Define analysis folder
analysis_folder = "md_thermo"

# Define analysis ensemble
ensemble = "01_npt"  

# Properties to extract
properties = ["Density", "Volume", "Enthalpy"]

# Define GROMACS command
command = "energy"

# Special args to gromacs analysis
args = [ ]

# XVG output name
output_name = "properties" 

# Submit analysis on cluster
on_cluster = False

# Perform extraction from gromacs
extract = True

# Template for extraction
extract_template = "../../templates/bash/extract_properties_pbs.sh"

# Percentage to discard from beginning of the simulation
time_fraction = 0.25


gromacs_setup.analysis_extract_properties( 
    analysis_folder = analysis_folder,
    ensemble = ensemble,
    extracted_properties = properties, 
    command = command,
    args = args,
    output_name = output_name,
    on_cluster = on_cluster,
    extract = extract,
    extract_template = extract_template,
    time_fraction = time_fraction 
)